## Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.


In [ ]:
import numpy as np
import pandas as pd 
import scipy.sparse
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Load the data

Let's load the test data. This dataset lists many pairs of IDs that represent images of various classes. Our goal is to predict which pairs of IDs have the same class.

We don't have any training data here, nor do we have any features for the test objects. All we need to solve this task is the file with the indices for the pairs that we need to compare.


In [ ]:
test = pd.read_csv('./test_pairs.csv')
test.head(5)

In [ ]:
print(test.shape)

## Pairs

We have 26,325 IDs, but not all possible pairs are in the test set. Otherwise we would have 26,325 * 26,324 = 69,2979,300 pairs. Is it possible that the pairing wasn't random?


In [ ]:
unique_ids = test['FirstId'].to_list() + test['SecondId'].to_list()
unique_ids = np.array(unique_ids, dtype=int)
print('distinct IDs: {} - from {} to {}'.format(np.unique(unique_ids).shape[0], unique_ids.min(), unique_ids.max()))


After submitting the dummy predictions below, we get a score of 0.5. This means that exactly half of the pairs are coming from the same class: the organizers wanted to create a balanced dataset, which means the pairing is not random. If we can figure out the methodology used, we will get a significant boost in scoring.


In [ ]:
# dummy predictions
one_preds = test[['pairId']].copy(deep=True)
one_preds['Prediction'] = 1
one_preds.to_csv('./test_one_preds.csv', index=False)


## Occurrences

Some IDs appear more in the first column than others. But the unique IDs are distributed evenly across the range of appeareances. The conclusion is similar for the second column, but with less consistency. 


In [ ]:
# count of images per number of occurrences in the FirstId column
test['FirstId'].value_counts().value_counts().sort_index().to_frame().T

In [ ]:
# count of images per number of occurrences in the SecondId column
test['SecondId'].value_counts().value_counts().sort_index().to_frame().T

In [ ]:
# the number of appeareances in the first and second column are correlated
counts = test['FirstId'].value_counts().to_frame().join(test['SecondId'].value_counts())
counts['count'] = 1
counts.groupby(['FirstId', 'SecondId']).count().T


All IDs split into two buckets: the ones that appear 21 times and the ones that appear 36 times.
+ the max split for the ones appearing 21 times is 7-14.
+ the max split for the ones appearing 36 times is 15-21.


In [ ]:
# occurences
occurences = pd.Series(test['FirstId'].to_list() + test['SecondId'].to_list(), name='agg')
display(occurences.value_counts().value_counts())
display(occurences.value_counts().value_counts() / 1755)


## Pairs of images

In [ ]:
# flip df to get all pairs (x,y) and (y,x)
test_flip = test[['SecondId', 'FirstId']]
test_flip.columns = ['FirstId', 'SecondId']

# merge flipped df with initial df to get all pairs; remove dupes
test_pairs = pd.concat((test[['FirstId', 'SecondId']], test_flip), ignore_index=True).drop_duplicates()

# total number of unique pairs
print(test_pairs.shape)


In [ ]:
# if we remove dupes, we almost get the same picture (with some rare edge cases)
test_pairs['FirstId'].value_counts().value_counts()


In [ ]:
# NOTE: this takes a long while to run

# list
counts_pairs = []

# starting ID
starting_ID = 0

for starting_ID in range(26325):
    # IDs directly linked to a given ID
    idX = test_pairs.loc[test_pairs['FirstId']==starting_ID, 'SecondId'].to_list() + [starting_ID]

    # IDs indirectly linked to starting_ID
    idX_graph = test_pairs.loc[test_pairs['FirstId'].isin(idX)]
    idX_graph_list = np.unique(np.array(idX_graph['FirstId'].tolist() + idX_graph['SecondId'].tolist()))

    # pairs & unique IDs
    counts_pairs.append((idX_graph.shape[0], idX_graph_list.shape[0]))

counts_pairs_id = pd.DataFrame(counts_pairs, columns=['pairs', 'IDs'])


## Finding patterns in the pairing process

We will try to find patterns in the sampling method by using an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix), treating pairs `(FirstId, SecondId)` as edges in an undirected graph. 

_Note: incidence matrices are typically very sparse with huge dimensions, so it's best to use specific tools like `scipy.sparse` to manipulate them. More information can be found in [wikipedia](https://en.wikipedia.org/wiki/Sparse_matrix) and the [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html)._


In [ ]:
# rows & cols (scipy goes crazy if a matrix is indexed with pandas' series, so we transform them into numpy arays)
rows = test_pairs['FirstId'].to_numpy()
cols = test_pairs['SecondId'].to_numpy()

# sparse matrix
inc_mat = scipy.sparse.coo_matrix(([1] * len(rows), (rows, cols)))

# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872


In [ ]:
# mathematical operations are faster in csr format
inc_mat_csr = inc_mat.tocsr()


In [ ]:
# plot pairs
#fig, ax = plt.subplots(figsize=(22,22))
#plt.spy(inc_mat_csr, markersize=0.1)


## Measure how close pairing lists are 

Each row of the incidence matrix represents the pairs that exist for a given image. We can measure how close two IDs are from one another by comparing how similar their representation is, i.e. how similar their pairings are.

For each pair of images, we compare how much overlap exists between their pairing list. We use the dot product to do so.


In [ ]:
# representations of all IDs
rows_FirstId  = inc_mat_csr[test['FirstId'].to_numpy()]
rows_SecondId  = inc_mat_csr[test['SecondId'].to_numpy()]


In [ ]:
# measure the overlap between the pairing list of each pair
f = np.array(rows_FirstId.multiply(rows_SecondId).sum(axis=1)).flatten()

# Sanity check
assert f.shape == (368550, )


## Convert to binary predictions

We know that half the pairs belong to the same class. We will find the threshold that splits our predictions into equal buckets.

_Note: the more overlap, the likelier it is that two images belong to the same class._


In [ ]:
# frequencies of each number of shared comparison IDs 
unique, counts = np.unique(f, return_counts=True)
print(np.asarray((unique, counts)).T)


In [ ]:
# we choose the value that splits the dataset in roughly equal buckets
pred = f >= 20

# final predictions
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)
submission.to_csv('submission.csv', index=False)


Go to the [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.

If you did everything right, the score should be very high.

## Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!
